<a href="https://colab.research.google.com/github/MarinaMamys/public_colab/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22Project4_Mamisheva_M_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project № 4

---

**Цель**

Подготовить основу рекомендательной системы в онлайн-школе MasterMind, благодаря которой можно будет предлагать клиентам интересные им курсы и тем самым повышать средний чек.

---
**Задачи**
1. Подготовить, проанализировать и обработать исходные данные.
2. Составить итоговую таблицу с рекомендациями. Она должна состоять из трёх столбцов:




*   Курс, к которому идёт рекомендация.
*   Курс для рекомендации № 1 (самый популярный).
*   Курс для рекомендации № 2 (второй по популярности).



В качестве исходных данных две таблицы, которые находятся в схеме final в Metabase.


*   carts с данными о пользовательских корзинах (дате создания, статусе, id пользователя-владельца и т. д.);
*   cart items с данными о курсах, которые пользователи добавили в корзину.

***Задача 1.*** На этапе подготовки данных напишем SQL-запросы, чтобы выгрузить данные по всем продажам курсов в разрезе пользователей.






---


Посмотрим, сколько всего уникальных пользователей добавляли что-либо в корзину. SQL-запрос:
> SELECT count(DISTINCT c.user_id) <br>
FROM final.carts c

Ответ: 70254 человека.



---


Узнаем, сколько из них покупали курсы?
SQL-запрос:
>SELECT count(DISTINCT c.user_id)<br>
>FROM final.carts c <br>
>RIGHT JOIN final.cart_items ci ON c.id=ci.cart_id <br>
>WHERE c.state = 'successful' AND ci.resource_type='Course'

Всего 49006 клиентов осуществили покупки курсов. Это около 70% из всех пользователей. Хороший показатель!




---


Чтобы узнать, сколько всего есть различных курсов, напишем SQL-запрос:
>SELECT count(DISTINCT ci.resource_id) <br>
>FROM final.cart_items ci <br>
>WHERE ci.resource_type='Course'

Всего 127 курсов.




---


Посмотрим, сколько всего из них покупали клиенты. SQL-запрос:
>SELECT count(DISTINCT ci.resource_id) <br>
FROM final.cart_items ci <br>
RIGHT JOIN final.carts c ON c.id=ci.cart_id <br>
WHERE (c.state = 'successful')
AND ( ci.resource_type='Course') <br>
>ORDER BY 1

Всего 126 курсов из оплаченных. Выходит, 1 курс вообще ни разу не купили. Посмотрим, какой это курс. SQL-запрос:
>SELECT DISTINCT ci.resource_id <br>
FROM final.cart_items ci <br>
WHERE ci.resource_type='Course' <br>

>EXCEPT <br>

>(SELECT DISTINCT ci.resource_id <br>
FROM final.cart_items ci <br>
RIGHT JOIN final.carts c ON c.id=ci.cart_id <br>
WHERE (c.state = 'successful') <br>
    AND ( ci.resource_type='Course'))

Ответ: ID курса, который не купили ни разу -772. Почему его никто не покупает?Можно посмотреть корзины пользователей. Если он в них есть, то не все потеряно. Возможно, над ним стоит поработать. Если такого курса нет в списках понравившихся, и проведена рекламная кампания, чтобы пользователи могли его заметить, то, возможно, стоит его убрать из предлагаемых курсов. Рекомендуется провести дополнительное исследование по этому курсу.  




---


Интересно, каково среднее число купленных курсов на одного клиента?
SQL-запрос:
>SELECT avg(count_resource_id) <br>
>FROM <br>
>> (SELECT count(ci.resource_id) count_resource_id <br>
  FROM final.carts c <br>
  RIGHT JOIN final.cart_items ci ON c.id=ci.cart_id <br>
  WHERE c.state = 'successful' <br>
   AND ci.resource_type='Course' <br>
   GROUP BY c.user_id) sub

Ответ: 1.44.
Получается, что в среднем пользователь выбирает больше 1 курса. Это хорошо. Посчитаем, сколько клиентов купили больше одного курса.

>SELECT count(user_id) count_user_id <br>
>FROM <br>
>> (SELECT c.user_id <br>
   FROM final.carts c <br>
   RIGHT JOIN final.cart_items ci ON c.id=ci.cart_id <br>
   WHERE c.state = 'successful' <br>
     AND ci.resource_type='Course' <br>
   GROUP BY c.user_id <br>
   HAVING count(DISTINCT ci.resource_id)>1) <br>

Ответ: 12656 человек. В результате, около 26 % из всех платящих за курсы клиентов, выбрали более 1 курса.  


---


Возьмем таких пользователей за основу для анализа и посмотрим, какие курсы они решаются оплатить.
Для этого выберем из исходных таблиц carts и cart_items  пользователей, которые приобрели более 1 курса, а после выведем для них все оплаченные учебные курсы.

In [ ]:
#Импортируем все необходимые библиотеки
import psycopg2
from psycopg2 import extras as pe
import pandas as pd
import numpy as np
import pprint
from pprint import pprint
import collections
from collections import Counter
import itertools
from itertools import combinations




In [ ]:
#Создадим функцию get_users_courses(), в которой выполним SQL-запрос:
def get_users_courses():
    query= '''
WITH subguery_users AS --в подзапросе найдем пользователей, купивших более одного курса

  (SELECT c.user_id --выберем пользователей
   FROM final.carts c --объединим две исходные таблицы по ключу-идентификатор корзины
   RIGHT JOIN final.cart_items ci ON c.id=ci.cart_id
   WHERE c.state = 'successful' --фильтр по успешному статусу состояния оплаты
     AND ci.resource_type='Course' --фильтр по типу продукта- курс
   GROUP BY c.user_id --группировка по ID пользователя
   HAVING count(DISTINCT ci.resource_id)>1) --условие отбора-количество курсов у 1 пользователя должно быть более 1

--выберем пользователей и их оплаченные курсы
SELECT DISTINCT user_id,
                resource_id
FROM final.carts c --объединим две исходные таблицы по ключу-идентификатор корзины
RIGHT JOIN final.cart_items ci ON c.id=ci.cart_id
WHERE (c.state = 'successful')--фильтр по статусу состояния оплаты
  AND (ci.resource_type='Course')--фильтр по типу продукта-курс
  AND (user_id in --фильтр по пользователям, купившим более 1 курса
         (SELECT *
          FROM subguery_users))
GROUP BY 1,2 --группировка по ID пользователей и ID курсов
ORDER BY 1 --сортировка по ID пользователей
'''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

#Выполним функцию запроса и запишем полученные данные в датафрейм df.
df = pd.DataFrame(get_users_courses())

In [ ]:
#Посмотрим на таблицу
display(df)

#проверим структуры таблицы, и есть ли пустые значения
display(df.info())

,user_id,resource_id
0,51,516
1,51,1099
2,6117,356
3,6117,357
4,6117,1125
...,...,...
34069,2188926,515
34070,2188926,743
34071,2190141,756
34072,2190141,794


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34074 entries, 0 to 34073
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   user_id      34074 non-null  int64
 1   resource_id  34074 non-null  int64
dtypes: int64(2)
memory usage: 532.5 KB


None

Всего 34074 записи, пустых нет, типы данных подходят.





In [ ]:
#проверим количество разных курсов
print(df['resource_id'].nunique())

#проверим количество уникальных клиентов
print(df['user_id'].nunique())


126
12656


Количество разных курсов-126- такое же, как мы вычислили раньше. Количество уникальных клиентов- 12656-осталось такое же. Нет утерянных данных. Можно приступить к созданию таблицы с рекомендациями.

***Задача 2.*** Для формирования итоговой таблицы сначала узнаем, какие курсы чаще всего покупают пользователи вместе с конкретным курсом.

Затем проранжируем все курсы, которые предлагает компания, и выберем только те, что с наибольшей вероятностью заинтересуют клиентов.

In [ ]:
#сгруппируем курсы по пользователю и запишем уникальные курсы для каждого пользователя в список
group_df=df.groupby('user_id')['resource_id'].apply(lambda x: list(np.unique(x))).reset_index()
group_df

,user_id,resource_id
0,51,"[516, 1099]"
1,6117,"[356, 357, 1125]"
2,10275,"[553, 1147]"
3,10457,"[361, 1138]"
4,17166,"[356, 357]"
...,...,...
12651,2179430,"[566, 750]"
12652,2186581,"[794, 864, 1129]"
12653,2187601,"[356, 553, 571, 765, 912]"
12654,2188926,"[515, 743]"


In [ ]:
#Сформируем для каждого пользователя пары курсов. Если у кого-то больше двух курсов, то пар будет несколько.
group_df['cours_pairs']=group_df['resource_id'].apply(lambda x: list(itertools.combinations(sorted (x),2)))

#посмотрим на результат
group_df

,user_id,resource_id,cours_pairs
0,51,"[516, 1099]","[(516, 1099)]"
1,6117,"[356, 357, 1125]","[(356, 357), (356, 1125), (357, 1125)]"
2,10275,"[553, 1147]","[(553, 1147)]"
3,10457,"[361, 1138]","[(361, 1138)]"
4,17166,"[356, 357]","[(356, 357)]"
...,...,...,...
12651,2179430,"[566, 750]","[(566, 750)]"
12652,2186581,"[794, 864, 1129]","[(794, 864), (794, 1129), (864, 1129)]"
12653,2187601,"[356, 553, 571, 765, 912]","[(356, 553), (356, 571), (356, 765), (356, 912..."
12654,2188926,"[515, 743]","[(515, 743)]"


In [ ]:
#выведем все пары курсов по всем пользователям в список total_pairs
total_pairs=[]
for i in range(len(group_df['cours_pairs'])):
  for j in range(len(group_df['cours_pairs'].iloc[i])):
               total_pairs.append(group_df['cours_pairs'].iloc[i][j])
#вот, что получилось
display(total_pairs)

[(516, 1099),
 (356, 357),
 (356, 1125),
 (357, 1125),
 (553, 1147),
 (361, 1138),
 (356, 357),
 (1125, 1140),
 (551, 745),
 (553, 745),
 (551, 1138),
 (553, 568),
 (514, 517),
 (514, 566),
 (517, 566),
 (363, 511),
 (363, 562),
 (363, 563),
 (511, 562),
 (511, 563),
 (562, 563),
 (568, 745),
 (509, 553),
 (509, 745),
 (553, 745),
 (1125, 1144),
 (509, 568),
 (509, 672),
 (568, 672),
 (516, 552),
 (356, 552),
 (357, 571),
 (568, 745),
 (509, 516),
 (509, 568),
 (516, 568),
 (513, 1141),
 (571, 1125),
 (551, 552),
 (551, 744),
 (551, 862),
 (551, 1138),
 (552, 744),
 (552, 862),
 (552, 1138),
 (744, 862),
 (744, 1138),
 (862, 1138),
 (361, 1138),
 (356, 679),
 (571, 745),
 (571, 1099),
 (745, 1099),
 (509, 516),
 (509, 568),
 (509, 1099),
 (516, 568),
 (516, 1099),
 (568, 1099),
 (517, 750),
 (800, 1125),
 (569, 840),
 (509, 568),
 (745, 1125),
 (509, 514),
 (509, 551),
 (509, 745),
 (514, 551),
 (514, 745),
 (551, 745),
 (571, 765),
 (745, 1187),
 (363, 566),
 (513, 1125),
 (1100, 1103

In [ ]:
#приведем список к объекту Counter, чтобы можно было посчитать для каждой пары количество ее вхождений в общий список пар

#создадим переменную-счетчик count_pairs
count_pairs=Counter(total_pairs)


In [ ]:
# с помощью функции most_common() получим самую популярную пару курсов, передав в качестве аргумента цифру 1:
display(count_pairs.most_common(1))


[((551, 566), 797)]

In [ ]:
#для дальнейшего анализа создадим словарь на основе счетчика
dict_course=dict(count_pairs)

#выведем словарь
pprint(dict_course)



{(356, 357): 100,
 (356, 360): 1,
 (356, 361): 17,
 (356, 366): 15,
 (356, 367): 12,
 (356, 368): 1,
 (356, 489): 26,
 (356, 490): 13,
 (356, 502): 17,
 (356, 503): 1,
 (356, 508): 1,
 (356, 509): 5,
 (356, 513): 1,
 (356, 514): 35,
 (356, 515): 21,
 (356, 516): 16,
 (356, 517): 3,
 (356, 519): 14,
 (356, 523): 24,
 (356, 551): 48,
 (356, 552): 7,
 (356, 553): 5,
 (356, 564): 3,
 (356, 566): 21,
 (356, 568): 6,
 (356, 569): 2,
 (356, 570): 1,
 (356, 571): 103,
 (356, 659): 4,
 (356, 671): 2,
 (356, 672): 4,
 (356, 679): 8,
 (356, 742): 1,
 (356, 743): 1,
 (356, 745): 8,
 (356, 749): 1,
 (356, 750): 3,
 (356, 753): 3,
 (356, 756): 5,
 (356, 757): 1,
 (356, 764): 8,
 (356, 765): 35,
 (356, 776): 2,
 (356, 777): 4,
 (356, 791): 1,
 (356, 794): 10,
 (356, 800): 3,
 (356, 803): 1,
 (356, 809): 2,
 (356, 829): 5,
 (356, 835): 1,
 (356, 840): 1,
 (356, 862): 3,
 (356, 863): 1,
 (356, 866): 1,
 (356, 907): 1,
 (356, 908): 3,
 (356, 909): 3,
 (356, 912): 34,
 (356, 1099): 3,
 (356, 1100): 5,
 (

In [ ]:
#посмотрим на значения словаря
c=pd.Series(dict_course.values())
print(c.describe())




count    3989.000000
mean       10.031838
std        26.355998
min         1.000000
25%         1.000000
50%         3.000000
75%         9.000000
max       797.000000
dtype: float64


Как видим, всего образовалось 3989 пар курсов с разными показателями частоты выбора пары. На среднее значение по частоте лучше посмотреть по медиане, так как существует сильный разброс  между минимальным значением - 1 раз и максимальным - 797 раз. В 50% случаев клиенты образовались пары, которые выбирают всего лишь 3 раза. Недалеко от минимального.
Посмотрим на 75 процентиль-  это тоже немного- всего 9 раз, но лучше, чем 3. Посчитаем для сравнения еще 80 процентиль.

In [ ]:
#посчитаем 80 процентиль

perc=np.percentile(c, [80])
pprint(perc)


array([11.])


>Используя 80 процентиль для того, чтобы отсечь менее популярные курсы, мы выберем только самые "сливки". <br>
Если использовать медиану со значением популярности-3, то в таблицу рекомендаций попадут и менее популярные курсы, но которые могут быть также интересны нашим клиентам. Что выбрать? <br>
Продакт-менеджер высказал пожелание, что "для начала нужно предлагать пару самых популярных курсов. Поэтому, в рамках данного проекта используем 80 процентиль. <br>
В дальнейшем, если будет стоять задача рекомендовать менее популярные или новые курсы, то сможем взять выборку по медиане.

Если пара курсов  встречается меньше, чем 11 раз, то для искомого курса нужно порекомендовать более популярный курс, который может быть ему понравиться.
Если пользователь выбирает каких-то 2 курса, значит ему интересны темы этих 2 курсов. Следовательно, если другой человек выбирает другую пару курсов,то ему интересны темы других 2 курсов. Если мы найдем столкновение интересов этих  пользователей, то, возможно, окажется, что и вторые курсы, которые они выбирали чаще в паре с искомым курсом, будут им полезны. Если в другой паре он встретится чаще, то мы с уверенностью сможем порекомендовать не сам курс(который выдал плохой спрос), а его пару.

В любом случае, мы должны посоветовать самые востребованные курсы, которые часто встречаются в паре. Так что это будет хорошая рекомендация.

Есть вероятность, что в результате подобного перебора значения ID рекомендательных курсов совпадут. В таком случае в качестве курса № 2 можно выбрать следующий по частоте встречаемости после максимального курс-пару на текущей итерации.

Если окажется, что и вспомогательная пара встречается редко, то нужно повторить цикл до тех пор, пока не найдется пара со значением в словаре больше 11.









Напишем функцию, которая сформирует список предпочтений для каждого курса. Список будет состоять из 2 кортежей, в каждом из которых 3 элемента: <br>

*   номер курса,
*   номер курса-рекомендация,
*   популярность курса-значению словаря dict-pairs по ключу-паре. <br>

Список будет отсортирован по популярности в порядке убывания. Так мы сможем для искомого курса извлечь 2 рекомендации: курс № 1- самый популярный(первый в списке) и курс №2 -второй по популярности.

In [ ]:
#функция, которая на входе принимает курс, к которому нужно выдать рекомендацию, и на выходе возвращает список рекомендаций
def recommend_func(resource_id):

  #создаем пустой список рекомендаций
  list_course_pairs=list()

  #формируем для каждого курса в паре ключей словаря кортеж из 3х элементов:(номер курса,номер курса-пары,значение словаря по ключу-паре) и добавляем его в список
  for key in dict_course.keys():
    if (key[0]==resource_id):
      list_course_pairs.append((resource_id, key[1],dict_course[key]))

    else:
      if (key[1]==resource_id):
        list_course_pairs.append((resource_id, key[0],dict_course[key]))


    #сортируем полученный список рекомендаций по популярности по убыванию
  sort_list_course_pairs=sorted(list_course_pairs, key= lambda x:  x[2] , reverse=True)

  #и выдаем первые 2 элемента в списке
  list_result=sort_list_course_pairs[:2]
  #print(list_result)


  #проверяем условие, что частота встречаемости в паре не должна быть меньше 11.
  #отработаем все возможные варианты
  if (list_result[0][2])<11 and (list_result[1][2])<11:
      #вновь вызываем функцию рекомендаций, подав на ее вход уже не сам курс, а его пару с частотой меньше 11 на выходе после первого запуска функции
      list_result[0]=recommend_func(list_result[0][1])[0]
      list_result[1]=recommend_func(list_result[1][1])[0]

  elif (list_result[0][2])>11 and (list_result[1][2])<11:
    #заменяем 2 элемент в списке на самый популярный курс в результате вызова функции, подав на ее вход курс, с которым исходный курс встретился менее 11 раз
      list_result[1]=recommend_func(list_result[1][1])[0]
      #print(list_result)

  elif (list_result[0][2])<11 and (list_result[1][2])>11:
    #заменяем 1 элемент в списке на самый популярный курс в результате вызова функции, подав на ее вход курс, с которым исходный курс встретился менее 11 раз
      list_result[0]=recommend_func(list_result[0][1])[0]

  #проверяем, совпадают ли номера курсов, которые окажутся в 1 и 2 столбцах
  if list_result[0][1]== list_result[1][1]:
    #если да, то для 2 столбца возьмем  из результата функции на текущей итерации второй по популярности курс-пару
    list_result[1]=recommend_func(list_result[1][0])[1]

  #возвращаем результирующий список
  return list_result



In [ ]:
#создадим пустую таблицу рекомендаций
recommend_df=pd.DataFrame()

#сформируем список уникальных курсов для 1 столбца итоговой таблицы рекомендаций, отсортируем по номеру курса.
unique_courses=sorted(list(df['resource_id'].unique()))

#заполним первый столбец с номерами курсов, для которых нужно сформировать рекомендации
recommend_df['resource_id']=pd.Series(unique_courses)

#заполним 2 столбец- курс для рекомендации № 1, применив функцию recommend_func ко всему 1 столбцу и взяв самый популярный курс
recommend_df['course_1_recommended']= recommend_df['resource_id'].apply(recommend_func).apply(lambda x: x[0][1])

#заполним 3 столбец- курс для рекомендации № 2, применив функцию recommend_func ко всему 1 столбцу и взяв второй по популярности
recommend_df['course_2_recommended']= recommend_df['resource_id'].apply(recommend_func).apply(lambda x: x[1][1])

In [ ]:
#посмотрим на таблицу
display(recommend_df)


,resource_id,course_1_recommended,course_2_recommended
0,356,571,357
1,357,571,356
2,358,570,752
3,359,570,358
4,360,745,516
...,...,...,...
121,1188,1141,513
122,1198,1125,566
123,1199,551,489
124,1200,551,752


In [ ]:
#у нас получилось! проверим количество уникальных курсов- их должно быть 126
display(recommend_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   resource_id           126 non-null    int64
 1   course_1_recommended  126 non-null    int64
 2   course_2_recommended  126 non-null    int64
dtypes: int64(3)
memory usage: 3.1 KB


None

Все верно! Пустых ячеек нет. Для каждого курса найдены рекомендации № 1 и №2.

In [ ]:
#Выгрузим результаты в файл recommend_table.csv.
recommend_df.to_csv('recommend_table.csv', index=False)

Конечно, можно было бы просто рекомендать к курсу-паре, у которой частота встречаемости меньше 11, самый популярный курс или курсы независимо от пары. Но такой вариант менее предпочтителен по следующим причинам:

1.  При таких рекомендациях  предпочтения пользователей не будут приняты во внимание совсем. В нашем случае учитывается максимальный интерес интереса.
2.  Продакт-менеджер Григорий при постановке задач упомянул о том, чтобы вызвать интерес среди пользователей и к другим курсам. Что мы и сделали в нашей рекомендательной таблице.
3.  Популярные курсы и так будут рекомендованы в приоритетном порядке.

## Выводы
В результате работы над данным проектом мы подготовили таблицу, которая состоит из трёх столбцов:

1.  Курс, к которому идёт рекомендация.
2.  Курс для рекомендации № 1.
3.  Курс для рекомендации № 2.

В ходе решения задачи мы сначала сформировали пары курсов, которые выбирают пользователи, затем определили для каждой пары частоту встречаемости среди всех пользователей.
После начали заполнять таблицу: к каждому курсу из 1 столбца находили пары, проранжировали их по убыванию по частоте, выбрали топ-2.
Далее самую популярную из этих пар к искомому курсу отнесли во 2й столбец.
А в 3й столбец- вторую по популярности пару.  

Отсекли из рекомендаций редко пользующиеся спросом курсы- те, что встречаются меньше 11 раз (80 процентиль) с искомым курсом в паре. Вместо них посоветовали  курсы, которые пользователи чаще выбирают в паре с его рекомендацией, которая также может быть ему интересна.

Такая таблица станет хорошей основой рекомендательной системы, которую можно в дальнейшем улучшать.  Она поможет повысить интерес пользователей и к другим курсам, расширить продуктовую корзину, тем самым увеличив средний чек.

>Для роста продаж и увеличения лояльности клиентов также желательно проанализировать, почему одни курсы- популярные, а другие-нет.  Возможно, стоит использовать полезные свойства/фичи востребованных курсов при разработке новых. На последние можно предоставлять бесплатный доступ, например, на пару дней, чтобы пользователи могли познакомиться с ними. Нужно принять во внимание, что в каких-то курсах достаточно отредактировать программу или наполнение, и они найдут своих клиентов, а от некоторых - придется отказаться совсем.


